In [ ]:
%matplotlib inline

import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import math
import pickle
import pandas as pd
import scipy.io
import time
import h5py
import bz2

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

from numpy import linalg as LA
from scipy.spatial import Delaunay
from sklearn.neighbors import NearestNeighbors

#sys.path.insert(0, "../")
from info3d import *
from nn_matchers import *

# EXTRACTING the existing sample data

In [ ]:
with open('point_collection/new_contiguous_point_collection.pickle','rb') as f: 
    new_contiguous_point_collection = pickle.load(f)
    
with open('descriptors/new_complete_res5_4by5_descriptors.pickle','rb') as f:
    descriptors = pickle.load(f)

"""
Parameters
"""
# We used a radius range of 0.25 to 5.0 in increments of 0.25
radius_range = np.arange(0.5,1.6,0.5)

# Extracting the bz2 to regular pickle
for radius in radius_range:
    
    with bz2.BZ2File('testing_samples/{}_successive_point_cloud.pickle.bz2'.format(radius), 'r') as bz2_f:
        successive_point_collection = pickle.load(bz2_f)
        
    with open('testing_samples/{}_successive_point_cloud.pickle'.format(radius), 'wb') as f:
        pickle.dump(successive_point_collection,f)
        
    with bz2.BZ2File('testing_samples/{}_partial_point_cloud.pickle.bz2'.format(radius), 'r') as bz2_f:
        partial_point_collection = pickle.load(bz2_f)
        
    with open('testing_samples/{}_partial_point_cloud.pickle'.format(radius), 'wb') as f:
        pickle.dump(partial_point_collection,f)
        

# Step 1.1: Testing with Partial Spaces 

# 1.1 Testing our partial samples: Raw spaces

In [ ]:
for radius in radius_range:
        
    t0 = time.time()
    
    with open('testing_samples/{}_partial_point_cloud.pickle'.format(radius), 'rb') as f:
        partial_point_collection = pickle.load(f)
        samples = len(partial_point_collection)
        
    partial_scores = []
    
    for obj_meta, partial_pointcloud, partial_triangles in partial_point_collection:
        
        # ROTATION
        random_theta =  (2*np.pi)*np.random.random()# from [0, 2pi)
        random_axis = np.random.choice(np.arange(0,3))
        rotated_pointCloud = rotatePointCloud(partial_pointcloud, random_theta, random_axis)

        # TRANSLATION
        t_pointCloud = np.asarray(rotated_pointCloud)
        random_tx_axis = np.random.choice(np.arange(0,3))
        random_translation = np.random.random()
        t_pointCloud[:,random_tx_axis] = t_pointCloud[:,random_tx_axis] + random_translation
        t_triangles = np.asarray(partial_triangles)#-vertices_length

        t1 = time.time()

        try:
            p_descriptors, p_keypoints, p_d_c = getSpinImageDescriptors(
                t_pointCloud,
                down_resolution = 5,
                cylindrical_quantization = [4,5]
            )
        except Exception as ex:
            print("Error getting descriptors of",obj_meta)
            print("Error Message:",ex)
            
            continue

        # Resetting the diff_Ratio matrix
        diff_scores = np.ones((p_descriptors.shape[0],len(descriptors),2))
        diff_ratios = np.ones((p_descriptors.shape[0],len(descriptors)))
        diff_indexs = np.ones((p_descriptors.shape[0],len(descriptors),2))

        #print(diff_ratios.shape)
        local_keypoint_matches = []

        for i_r, ref_descriptor in enumerate(descriptors):

            r_descriptors = ref_descriptor[1]
            r_keypoints = ref_descriptor[2]

            matching_range = np.arange(r_descriptors.shape[1])

            try:    
                f_nearestneighbor, diff = getMatches(p_descriptors,r_descriptors,2,range_to_match=matching_range)
                diff = diff/np.amax(diff) # max-normalization of differences
                diff_ratio = diff[:,0]/diff[:,1]
                diff_ratios[:,i_r] = diff_ratio
                diff_scores[:,i_r] = diff
                diff_indexs[:,i_r] = f_nearestneighbor
                
                # Taking note of the matched keypoints
                local_keypoint_matches.append([
                    obj_meta,
                    p_keypoints,
                    r_keypoints[f_nearestneighbor[:,0]]
                ])

            except Exception as ex:
                print(rotation,"Error Matching:",ex)

        # Accumulating the diff_ratio matrix for every partial (rotated) object
        partial_scores.append([
            obj_meta,
            np.asarray(diff_ratios),
            np.asarray(diff_indexs),
            np.asarray(diff_scores),
            local_keypoint_matches
        ])

        if len(partial_scores) % 10 == (samples-1)%10:
            #print('Test')
            print(" radius = {}: Done with {} iterations. Time to match {:.3f} seconds.".format(
                radius,
                len(partial_scores),
                time.time()-t0)
                 )
            t0 = time.time()
        
            current_errors = NN_matcher(partial_scores)
            print("   Error Rate:",np.sum(current_errors[:,1]/len(partial_scores)))

    partial_errors = NN_matcher(partial_scores)
    print(radius,"Error Rate:",np.sum(partial_errors[:,1]/len(partial_scores)))
                                                                                
    with open('testing_results/partials/radius_{}_RAW_scores.pickle'.format(radius), 'wb') as f:
        pickle.dump(partial_scores,f)
                                              
    with open('testing_results/partials/radius_{}_RAW_errors.pickle'.format(radius), 'wb') as f:
        pickle.dump(partial_errors,f)


# 1.2 Testing our partial samples: RANSAC spaces

In [ ]:
for radius in radius_range:
        
    t0 = time.time()
    
    with open('testing_samples/{}_partial_point_cloud.pickle'.format(radius), 'rb') as f:
        partial_point_collection = pickle.load(f)
        samples = len(partial_point_collection)
        
    partial_scores = []
    partial_properties = []
    
    for obj_meta, partial_pointcloud, partial_triangles in partial_point_collection:
        
        # ROTATION
        random_theta =  (2*np.pi)*np.random.random()# from [0, 2pi)
        random_axis = np.random.choice(np.arange(0,3))
        rotated_pointCloud = rotatePointCloud(partial_pointcloud, random_theta, random_axis)

        # TRANSLATION
        t_pointCloud = np.asarray(rotated_pointCloud)
        random_tx_axis = np.random.choice(np.arange(0,3))
        random_translation = np.random.random()
        t_pointCloud[:,random_tx_axis] = t_pointCloud[:,random_tx_axis] + random_translation
        t_triangles = np.asarray(partial_triangles)#-vertices_length

        #if object_name % 13 == 0:
        #    print('{}: Processing iteration {} of object {}.'.format(radius,iteration,object_name))

        t1 = time.time()

        # GETTING GENERALIZATION
        gen_planes = getRansacPlanes(
            t_pointCloud
        )

        p_pointcloud, p_triangles = getGeneralizedPointCloud(
            planes=gen_planes, 
        )

        try:
            p_descriptors, p_keypoints, p_d_c = getSpinImageDescriptors(
                p_pointcloud,
                down_resolution = 5,
                cylindrical_quantization = [4,5]
            )
        except Exception as ex:
            print("Error getting descriptors of",obj_meta)
            print("Error Message:",ex)
            
            continue

        # Resetting the diff_Ratio matrix
        diff_scores = np.ones((p_descriptors.shape[0],len(descriptors),2))
        diff_ratios = np.ones((p_descriptors.shape[0],len(descriptors)))
        diff_indexs = np.ones((p_descriptors.shape[0],len(descriptors),2))

        #print(diff_ratios.shape)
        local_keypoint_matches = []

        for i_r, ref_descriptor in enumerate(descriptors):

            #o_, r_descriptors, r_keypoints, r_d_c
            r_descriptors = ref_descriptor[1]
            r_keypoints = ref_descriptor[2]

            matching_range = np.arange(r_descriptors.shape[1])

            try:    
                f_nearestneighbor, diff = getMatches(p_descriptors,r_descriptors,2,range_to_match=matching_range)
                diff = diff/np.amax(diff) # max-normalization of differences
                diff_ratio = diff[:,0]/diff[:,1]
                diff_ratios[:,i_r] = diff_ratio
                diff_scores[:,i_r] = diff
                diff_indexs[:,i_r] = f_nearestneighbor
                
                # Taking note of the matched keypoints
                local_keypoint_matches.append([
                    obj_meta,
                    p_keypoints,
                    r_keypoints[f_nearestneighbor[:,0]]
                ])

            except Exception as ex:
                print(rotation,"Error Matching:",ex)

        # Accumulating the diff_ratio matrix for every partial (rotated) object
        partial_scores.append([
            obj_meta,
            np.asarray(diff_ratios),
            np.asarray(diff_indexs),
            np.asarray(diff_scores),
            local_keypoint_matches
        ])

        if len(partial_scores) % 10 == (samples-1)%10:
            #print('Test')
            print(" radius = {}: Done with {} iterations. Time to match {:.3f} seconds.".format(
                radius,
                len(partial_scores),
                time.time()-t0)
                 )
            t0 = time.time()
        
            current_errors = NN_matcher(partial_scores)
            print("   Error Rate:",np.sum(current_errors[:,1]/len(partial_scores)))

    partial_errors = NN_matcher(partial_scores)
    print(radius,"Error Rate:",np.sum(partial_errors[:,1]/len(partial_scores)))
                                                                                
    with open('testing_results/partials/radius_{}_RANSAC_scores.pickle'.format(radius), 'wb') as f:
        pickle.dump(partial_scores,f)
                                              
    with open('testing_results/partials/radius_{}_RANSAC_errors.pickle'.format(radius), 'wb') as f:
        pickle.dump(partial_errors,f)


# 1.3 Results of partial spaces

In [ ]:
fig=plt.figure(figsize=(9, 3))

RawNN = []
RansacGeneralizedNN = []

RawNN_intra_errors = []
RansacGeneralizedNN_intra_errors = []

for radius in radius_range:
    
    try:
        with open('testing_results/partials/radius_{}_RAW_errors.pickle'.format(radius), 'rb') as f:
            partial_errors = pickle.load(f)

        RawNN.append([
            radius,
            np.mean(partial_errors[:,1]),
            np.std(partial_errors[:,1]),
        ])
        
        correct_interspace_labels_idxs = np.where(partial_errors[:,1]==0)[0]

        intraspace_errors  = partial_errors[correct_interspace_labels_idxs,2]

        RawNN_intra_errors.append([
            radius,
            np.nanmean(intraspace_errors),
            np.nanstd(intraspace_errors)
        ])
        
    except:
        pass
    
    try:
        with open('testing_results/partials/radius_{}_RANSAC_errors.pickle'.format(radius), 'rb') as f:
            partial_errors = pickle.load(f)

        RansacGeneralizedNN.append([
            radius,
            np.nanmean(partial_errors[:,1]),
            np.nanstd(partial_errors[:,1]),
        ])

        correct_interspace_labels_idxs = np.where(partial_errors[:,1]==0)[0]

        intraspace_errors  = partial_errors[correct_interspace_labels_idxs,2]

        RansacGeneralizedNN_intra_errors.append([
            radius,
            np.nanmean(intraspace_errors),
            np.nanstd(intraspace_errors)
        ])
        
    except:
        pass
    
RansacGeneralizedNN = np.asarray(RansacGeneralizedNN)
RawNN = np.asarray(RawNN)

RawNN_intra_errors = np.asarray(RawNN_intra_errors)
RansacGeneralizedNN_intra_errors = np.asarray(RansacGeneralizedNN_intra_errors)

ax1 = fig.add_subplot(121) 

ax1.grid(alpha = 0.7)
ax1.set_ylim(-0.025,1.025)
ax1.set_xlim(radius_range[0]-0.25,radius_range[-1]+0.25)
markersize = 8

ax1.set_ylabel("INTER-space Privacy")
ax1.set_xlabel("Partial Radius")
#ax1.set_yticklabels(fontsize = 16)
#ax1.set_xticklabels(fontsize = 16)

ax1.plot(
    RawNN[:,0],RawNN[:,1],
    "-o",
    linewidth = 2,
    mew = 2,markersize = markersize,
    label = "Raw"
)
ax1.plot(
    RansacGeneralizedNN[:,0],RansacGeneralizedNN[:,1],
    "-s",
    linewidth = 2,
    mew = 2,markersize = markersize,
    label = "RANSAC"
)

ax1.legend(loc = "lower left")

ax2 = fig.add_subplot(122) 

ax2.grid(alpha = 0.7)
ax2.set_ylim(-0.25,10.25)
ax2.set_xlim(radius_range[0]-0.25,radius_range[-1]+0.25)

ax2.set_ylabel("INTRA-space Privacy")
ax2.set_xlabel("Partial Radius")
#ax2.set_yticklabels(fontsize = 16)
#ax2.set_xticklabels(fontsize = 16)

plt.minorticks_on()

ax2.plot(
    RawNN_intra_errors[:,0],
    RawNN_intra_errors[:,1], 
    linewidth = 2,
    marker = 'o',fillstyle = 'none',
    mew = 2,markersize = markersize,
    label = "Raw"
)

ax2.plot(
    RansacGeneralizedNN_intra_errors[:,0],
    RansacGeneralizedNN_intra_errors[:,1], 
    linewidth = 2, 
    marker = 's',fillstyle = 'none',
    mew = 2,markersize = markersize,
    label = "RANSAC"
)

ax2.legend(loc = "lower left");

# Step 2 Testing with successively released partial spaces

In [ ]:
"""
Parameters
"""
# We used a radius range of 0.25 to 5.0 in increments of 0.25.
radius_range = radius_range[:2]

# For our work, we orignally used 50 samples with further 100 successive releases for our investigation.
# Below are lower parameters, change as desired.
#samples = 50
#releases = 50

# For demonstration purposes, we skip testing some successive samples but we still accumulate them.
skip = 3

# 2.1 Testing the successive case: RAW and RANSAC

In [ ]:
for radius in radius_range:
        
    t0 = time.time()
    
    try:
        with open('testing_samples/{}_successive_point_cloud.pickle'.format(radius), 'rb') as f:
            successive_point_collection = pickle.load(f)
            samples = len(successive_point_collection)

    except:
        pass
    
    successive_scores = []
    successive_errors = []
    
    g_successive_scores = []
    g_successive_errors = []

    for obj_, growing_point_collection in successive_point_collection:
        
        iteration_scores = []
        
        g_iteration_scores = []
        
        # ROTATION param
        random_theta =  (2*np.pi)*np.random.random()# from [0, 2pi)
        random_axis = np.random.choice(np.arange(0,3))
        
        # TRANSLATION param
        random_tx_axis = np.random.choice(np.arange(0,3))
        random_translation = np.random.random()
        
        growing_point_cloud = []
        growing_p_point_cloud = []
        growing_p_triangles = []
        
        releases = len(growing_point_collection)
        release_count = 0
    
        for obj_meta, partial_pointcloud, partial_triangles in growing_point_collection:
            #i_obj, released_growing_point_collection in enumerate(growing_point_collection):

            rotated_pointCloud = rotatePointCloud(partial_pointcloud, random_theta, random_axis)

            # TRANSLATION
            t_pointCloud = np.asarray(rotated_pointCloud)
            t_pointCloud[:,random_tx_axis] = t_pointCloud[:,random_tx_axis] + random_translation
            t_triangles = np.asarray(partial_triangles)#-vertices_length
            
            #Regular Accumulation
            if len(growing_point_cloud) == 0:
                growing_point_cloud = t_pointCloud

            else:
                growing_point_cloud = np.concatenate(
                    (growing_point_cloud,t_pointCloud),
                    axis=0
                )
                
            #RANSAC generalizations
            if len(growing_p_point_cloud) == 0:
                gen_planes = getLOCALIZEDRansacPlanes(
                    pointCloud = t_pointCloud,
                    original_vertex = obj_meta[-1]
                )
            else:
                gen_planes = updatePlanesWithSubsumption(
                    new_pointCloud=t_pointCloud,
                    existing_pointCloud=growing_p_point_cloud,
                    planes_to_find = max(min(release_count,50),30),
                    #verbose=True
                )
            
            if len(gen_planes) == 0:
                print("No gen planes after release",release_count,growing_point_cloud.shape)
                continue

            try:
                updated_point_cloud, updated_triangles = getGeneralizedPointCloud(
                    planes = gen_planes,
                    triangle_area_threshold = 0.2,#2.0*np.amax(getTriangleAreas(partial_pointCloud, partial_triangles))
                    #verbose = True
                )
                growing_p_point_cloud = updated_point_cloud
                growing_p_triangles = updated_triangles
                
                #print(" Successful:",release_count,len(growing_p_point_cloud), len(growing_p_triangles),partial_pointCloud.shape)
            except Exception as ex:
                print("Error getting updated point cloud in release",release_count+1)
                print(" ",growing_p_point_cloud.shape, growing_p_triangles.shape,partial_pointCloud.shape)
                #print(ex)
                continue
                
            if len(growing_p_point_cloud) == 0:
                continue
                
            release_count += 1
            
            if release_count % skip != 1: # skip 
                continue
                
            #Regular Processing
            growing_point_cloud = np.unique(growing_point_cloud,axis=0)
            
            try:
                p_descriptors, p_keypoints, p_d_c = getSpinImageDescriptors(
                    growing_point_cloud,
                    down_resolution = 5,
                    cylindrical_quantization = [4,5]
                )
            except:
                p_descriptors = []
                p_keypoints = []

            # Resetting the diff_Ratio matrix
            diff_scores = np.ones((p_descriptors.shape[0],len(descriptors),2))
            diff_ratios = np.ones((p_descriptors.shape[0],len(descriptors)))
            diff_indexs = np.ones((p_descriptors.shape[0],len(descriptors),2))

            local_keypoint_matches = []

            for i_r, ref_descriptor in enumerate(descriptors):

                r_descriptors = ref_descriptor[1]
                r_keypoints = ref_descriptor[2]

                matching_range = np.arange(r_descriptors.shape[1])

                try:    
                    f_nearestneighbor, diff = getMatches(p_descriptors,r_descriptors,2,range_to_match=matching_range)
                    diff = diff/np.amax(diff) # max-normalization of differences
                    diff_ratio = diff[:,0]/diff[:,1]
                    diff_ratios[:,i_r] = diff_ratio
                    diff_scores[:,i_r] = diff
                    diff_indexs[:,i_r] = f_nearestneighbor

                    # Taking note of the matched keypoints
                    local_keypoint_matches.append([
                        obj_meta,
                        p_keypoints,
                        r_keypoints[f_nearestneighbor[:,0]]
                    ])

                except Exception as ex:
                    print(rotation,"Error Matching:",ex)

            # Accumulating the diff_ratio matrix for every partial (rotated) object
            iteration_scores.append([
                obj_meta,
                diff_ratios,
                diff_indexs,
                diff_scores,
                local_keypoint_matches
            ])

            if release_count % 2 == 0:
                #print('Test')
                print("  radius = {}: Done with {} releases ({}) ({} samples). Time to match {:.3f} seconds. ({})".format(
                    radius,
                    len(iteration_scores),
                    release_count,
                    len(successive_scores),
                    time.time()-t0,
                    growing_point_cloud.shape
                )
                     )
                t0 = time.time()

                current_errors = NN_matcher(iteration_scores)
                print("   Error Rate:",np.sum(current_errors[:,1]/len(iteration_scores)))
            
            #RANSAC Processing         
            try:
                p_descriptors, p_keypoints, p_d_c = getSpinImageDescriptors(
                    growing_p_point_cloud,
                    down_resolution = 5,
                    cylindrical_quantization = [4,5]
                )
            except:
                p_descriptors = []
                p_keypoints = []
                
                print("Error getting descriptors at release",release_count,"; using next release as this release.")
                
                #release_count -= 1

                continue

            # Resetting the diff_Ratio matrix
            diff_scores = np.ones((p_descriptors.shape[0],len(descriptors),2))
            diff_ratios = np.ones((p_descriptors.shape[0],len(descriptors)))
            diff_indexs = np.ones((p_descriptors.shape[0],len(descriptors),2))

            local_keypoint_matches = []

            for i_r, ref_descriptor in enumerate(descriptors):

                #o_, r_descriptors, r_keypoints, r_d_c
                r_descriptors = ref_descriptor[1]
                r_keypoints = ref_descriptor[2]

                matching_range = np.arange(r_descriptors.shape[1])

                try:    
                    f_nearestneighbor, diff = getMatches(p_descriptors,r_descriptors,2,range_to_match=matching_range)
                    diff = diff/np.amax(diff) # max-normalization of differences
                    diff_ratio = diff[:,0]/diff[:,1]
                    diff_ratios[:,i_r] = diff_ratio
                    diff_scores[:,i_r] = diff
                    diff_indexs[:,i_r] = f_nearestneighbor

                    # Taking note of the matched keypoints
                    local_keypoint_matches.append([
                        obj_meta,
                        p_keypoints,
                        r_keypoints[f_nearestneighbor[:,0]]
                    ])

                except Exception as ex:
                    print(rotation,"Error Matching:",ex)

            # Accumulating the diff_ratio matrix for every partial (rotated) object
            g_iteration_scores.append([
                obj_meta,
                diff_ratios,
                diff_indexs,
                diff_scores,
                local_keypoint_matches
            ])

            if release_count % 2 == 0:
                #print('Test')
                print("  radius = {} [G]: Done with {} releases ({}) ({} samples). Time to match {:.3f} seconds. ({})".format(
                    radius,
                    len(g_iteration_scores),
                    release_count,
                    len(g_successive_scores),
                    time.time()-t0,
                    growing_p_point_cloud.shape
                )
                     )
                t0 = time.time()

                current_errors = NN_matcher(g_iteration_scores)
                print("   G_Error Rate:",np.sum(current_errors[:,1]/len(g_iteration_scores)))

        iteration_errors = NN_matcher(iteration_scores)

        g_iteration_errors = NN_matcher(g_iteration_scores)

        if len(successive_scores) % 5 == (samples-1)%5 :
            try:
                print(radius,len(successive_scores),"Error Rate:",np.sum(iteration_errors[:,1]/len(iteration_scores)))
                print(radius,len(g_successive_scores),"G_Error Rate:",np.sum(g_iteration_errors[:,1]/len(g_iteration_scores)))
            except:
                pass
            
        successive_scores.append([
            obj_,
            iteration_scores
        ])
        
        successive_errors.append([
            obj_,
            iteration_errors
        ])
        
        g_successive_scores.append([
            obj_,
            g_iteration_scores
        ])
        
        g_successive_errors.append([
            obj_,
            g_iteration_errors
        ])
    
        with open('testing_results/successive/radius_{}_RAW_successive_scores.pickle'.format(radius), 'wb') as f:
            pickle.dump(successive_scores,f)

        with open('testing_results/successive/radius_{}_RAW_successive_errors.pickle'.format(radius), 'wb') as f:
            pickle.dump(successive_errors,f)

        with open('testing_results/successive/radius_{}_RANSAC_successive_scores.pickle'.format(radius), 'wb') as f:
            pickle.dump(g_successive_scores,f)

        with open('testing_results/successive/radius_{}_RANSAC_successive_errors.pickle'.format(radius), 'wb') as f:
            pickle.dump(g_successive_errors,f)

# 2.2 Results of the successive case

In [ ]:
succ_RawNN_errors = []
succ_RawNN_partial_errors = []

succ_RansacGeneralizedNN_errors = []
succ_RansacGeneralizedNN_partial_errors = []

t0 = time.time()

for radius in radius_range:
    
    succ_RawNN_per_iteration_errors = []
    succ_RansacGeneralizedNN_per_iteration_errors = []

    try:
                
        with open('testing_results/successive/radius_{}_RAW_successive_scores.pickle'.format(radius), 'rb') as f:
            successive_scores = pickle.load(f)

        with open('testing_results/successive/radius_{}_RAW_successive_errors.pickle'.format(radius), 'rb') as f:
            successive_errors = pickle.load(f)
        
        for obj_, iteration_errors in successive_errors:    
            #print("  RAW",radius,iteration_errors.shape)

            if iteration_errors.shape[0] < int(releases/skip):
                continue
            else:
                succ_RawNN_per_iteration_errors.append(iteration_errors[:int(releases/skip)])
       
        succ_RawNN_errors.append([
            radius,
            np.asarray(succ_RawNN_per_iteration_errors)
        ])
        
        print(np.asarray(succ_RawNN_per_iteration_errors).shape)

    except:
        pass
    
    try:
                
        with open('testing_results/successive/radius_{}_RANSAC_successive_scores.pickle'.format(radius), 'rb') as f:
            successive_scores = pickle.load(f)

        with open('testing_results/successive/radius_{}_RANSAC_successive_errors.pickle'.format(radius), 'rb') as f:
            successive_errors = pickle.load(f)
        
        for obj_, iteration_scores in successive_scores:#[:-1]:    
            #print("  RANSAC",radius,iteration_errors.shape)
            iteration_errors = NN_matcher(iteration_scores)

            if iteration_errors.shape[0] < int(releases/skip):
                continue
            else:
                succ_RansacGeneralizedNN_per_iteration_errors.append(iteration_errors[:int(releases/skip)])
       
        succ_RansacGeneralizedNN_errors.append([
            radius,
            np.asarray(succ_RansacGeneralizedNN_per_iteration_errors)
        ])
        
        print(np.asarray(succ_RansacGeneralizedNN_errors).shape)

    except:# Exception as ex:
        #print(radius,": successive RansacNN\n  ", ex)
        pass
    
    print("Done with radius = {:.2f} in {:.3f} seconds".format(radius,time.time() - t0))
    t0 = time.time()
    
for radius, per_iteration_errors in succ_RawNN_errors:

    #print(per_iteration_errors.shape)

    succ_RawNN_partial_errors_per_rel = []

    for rel_i in np.arange(per_iteration_errors.shape[1]):

        correct_interspace_labels_idxs = np.where(per_iteration_errors[:,rel_i,1]==0)[0]

        intraspace_errors  = per_iteration_errors[correct_interspace_labels_idxs,rel_i,2]

        succ_RawNN_partial_errors_per_rel.append([
            rel_i,
            np.mean(intraspace_errors),
            np.std(intraspace_errors)
        ])

    succ_RawNN_partial_errors.append([
        radius,
        np.asarray(succ_RawNN_partial_errors_per_rel)
    ])
    
for radius, per_iteration_errors in succ_RansacGeneralizedNN_errors:

    #print(radius,per_iteration_errors.shape)

    succ_RansacGeneralizedNN_errors_per_rel = []

    for rel_i in np.arange(per_iteration_errors.shape[1]):

        correct_interspace_labels_idxs = np.where(per_iteration_errors[:,rel_i,1]==0)[0]

        intraspace_errors  = per_iteration_errors[correct_interspace_labels_idxs,rel_i,2]

        succ_RansacGeneralizedNN_errors_per_rel.append([
            rel_i,
            np.mean(intraspace_errors),
            np.std(intraspace_errors)
        ])

    succ_RansacGeneralizedNN_partial_errors.append([
        radius,
        np.asarray(succ_RansacGeneralizedNN_errors_per_rel)
    ])

In [ ]:
fig=plt.figure(figsize=(15, 5))

ax1 = fig.add_subplot(121) 

ax1.grid(alpha = 0.7)
ax1.set_ylim(-0.025,1.025)
ax1.set_xlim(0,releases-skip)
markersize = 8

ax1.set_ylabel("INTER-space Privacy")
ax1.set_xlabel("Releases")

for radius, RawNN_per_iteration_errors in succ_RawNN_errors:
    print(RawNN_per_iteration_errors.shape)
    ax1.plot(
        np.arange(1,releases-skip,skip),#[:RawNN_per_iteration_errors.shape[1]],
        np.mean(RawNN_per_iteration_errors[:,:,1], axis = 0), 
        ':o',
        label = "r ="+ str(radius) + " Raw"
    )
    
for radius, RansacNN_per_iteration_errors in succ_RansacGeneralizedNN_errors:
    print(RansacNN_per_iteration_errors.shape)
    ax1.plot(
        np.arange(1,releases-skip,skip),
        np.mean(RansacNN_per_iteration_errors[:,:,1], axis = 0),
        '-s',
        label = "r ="+ str(radius) + " RANSAC"
    )

ax1.legend(loc = "best", ncol = 2)

ax2 = fig.add_subplot(122) 

ax2.grid(alpha = 0.7)
ax2.set_ylim(-0.25,12.25)
ax2.set_xlim(0,releases-skip)

ax2.set_ylabel("INTRA-space Privacy")
ax2.set_xlabel("Partial Radius")
#ax2.set_yticklabels(fontsize = 16)
#ax2.set_xticklabels(fontsize = 16)

#plt.minorticks_on()

for radius, errors_per_rel in succ_RansacGeneralizedNN_partial_errors:
    ax2.plot(
        np.arange(1,releases-skip,skip),
        errors_per_rel[:,1], 
        #errors_per_rel[:,2],
        '-s',
        linewidth = 2, #capsize = 4.0, 
        #marker = markers[0],
        #fillstyle = 'none',
        mew = 2, markersize = markersize,
        label = "r ="+ str(radius)+", RANSAC"
    )

ax2.legend(loc = "best")

# Step 3.1 Testing with conservative plane releasing

In [ ]:
"""
Parameters:

Also, we use the same successive samples from successive releasing for direct comparability of results.
"""
# We used a radius range of 0.25 to 5.0 in increments of 0.25.
radius_range = radius_range[:2]

# For our work, we orignally used 50 samples with further 100 successive releases for our investigation.
# Below are lower parameters, change as desired.
samples = 25
releases = 50

planes = np.arange(1,30,3)

# For demonstration purposes, we skip testing some successive samples but we still accumulate them.
skip = 3

In [ ]:
for radius in radius_range:
        
    t0 = time.time()
    
    try:
        with open('testing_samples/{}_successive_point_cloud.pickle'.format(radius), 'rb') as f:
            successive_point_collection = pickle.load(f)
            samples = len(successive_point_collection)

    except:
        pass
    
    conservative_scores = []
    conservative_errors = []

    for obj_, growing_point_collection in successive_point_collection:
        
        # ROTATION param
        random_theta =  (2*np.pi)*np.random.random()# from [0, 2pi)
        random_axis = np.random.choice(np.arange(0,3))
        
        # TRANSLATION param
        random_tx_axis = np.random.choice(np.arange(0,3))
        random_translation = np.random.random()
        
        per_plane_scores = []
        per_plane_errors = []
        
        latest_plane = 0
        
        for plane_number in planes:
            
            print(radius," {} planes, ({} Done)".format(plane_number,len(per_plane_scores)))
        
            g_iteration_scores = []

            growing_point_cloud = []
            growing_p_point_cloud = []
            growing_p_triangles = []

            release_count = 0
            
            for obj_meta, partial_pointcloud, partial_triangles in growing_point_collection:
                #i_obj, released_growing_point_collection in enumerate(growing_point_collection):

                releases = len(growing_point_collection)

                # ROTATION
                rotated_pointCloud = rotatePointCloud(partial_pointcloud, random_theta, random_axis)

                # TRANSLATION
                t_pointCloud = np.asarray(rotated_pointCloud)
                t_pointCloud[:,random_tx_axis] = t_pointCloud[:,random_tx_axis] + random_translation
                t_triangles = np.asarray(partial_triangles)#-vertices_length

                #RANSAC generalizations
                try:
                    if len(growing_p_point_cloud) == 0:
                        gen_planes = getLOCALIZEDRansacPlanes(
                            pointCloud = t_pointCloud,#np.where(hPointConnectivity < 5)[0],axis = 0),
                            original_vertex = obj_meta[-1],
                            planes_to_find=plane_number
                            #verbose=True
                            #threshold = threshold # the point-plane distance threshold
                        )
                    else:
                        gen_planes = updatePlanesWithSubsumption(
                            new_pointCloud=t_pointCloud,
                            existing_pointCloud=growing_p_point_cloud,
                            planes_to_find = plane_number
                            #verbose=True
                        )
                except:
                    print("Error getting planes",release_count,growing_point_cloud.shape)
                    continue

                if len(gen_planes) == 0:
                    print("No gen planes after release",release_count,growing_point_cloud.shape)
                    continue

                try:
                    updated_point_cloud, updated_triangles = getGeneralizedPointCloud(
                        planes = gen_planes,
                        triangle_area_threshold = 0.2,#2.0*np.amax(getTriangleAreas(partial_pointCloud, partial_triangles))
                        #verbose = True
                    )
                    growing_p_point_cloud = updated_point_cloud
                    growing_p_triangles = updated_triangles

                    #print(" Successful:",release_count,len(growing_p_point_cloud), len(growing_p_triangles),partial_pointCloud.shape)
                except Exception as ex:
                    print("Error getting updated point cloud in release",release_count+1)
                    print(" ",growing_p_point_cloud.shape, growing_p_triangles.shape,partial_pointCloud.shape)
                    #print(ex)
                    continue

                if len(growing_p_point_cloud) == 0:
                    continue

                release_count += 1

                if release_count % skip != 1: # skip 
                    continue

                #RANSAC Processing         
                try:
                    p_descriptors, p_keypoints, p_d_c = getSpinImageDescriptors(
                        growing_p_point_cloud,
                        down_resolution = 5,
                        cylindrical_quantization = [4,5]
                    )
                except:
                    p_descriptors = []
                    p_keypoints = []

                    print("Error getting descriptors at release",release_count,"; using next release as this release.")

                    #release_count -= 1

                    continue

                # Resetting the diff_Ratio matrix
                diff_scores = np.ones((p_descriptors.shape[0],len(descriptors),2))
                diff_ratios = np.ones((p_descriptors.shape[0],len(descriptors)))
                diff_indexs = np.ones((p_descriptors.shape[0],len(descriptors),2))

                local_keypoint_matches = []

                for i_r, ref_descriptor in enumerate(descriptors):

                    #o_, r_descriptors, r_keypoints, r_d_c
                    r_descriptors = ref_descriptor[1]
                    r_keypoints = ref_descriptor[2]

                    matching_range = np.arange(r_descriptors.shape[1])

                    try:    
                        f_nearestneighbor, diff = getMatches(p_descriptors,r_descriptors,2,range_to_match=matching_range)
                        diff = diff/np.amax(diff) # max-normalization of differences
                        diff_ratio = diff[:,0]/diff[:,1]
                        diff_ratios[:,i_r] = diff_ratio
                        diff_scores[:,i_r] = diff
                        diff_indexs[:,i_r] = f_nearestneighbor

                        # Taking note of the matched keypoints
                        local_keypoint_matches.append([
                            obj_meta,
                            p_keypoints,
                            r_keypoints[f_nearestneighbor[:,0]]
                        ])

                    except Exception as ex:
                        print(rotation,"Error Matching:",ex)

                # Accumulating the diff_ratio matrix for every partial (rotated) object
                g_iteration_scores.append([
                    obj_meta,
                    diff_ratios,
                    diff_indexs,
                    diff_scores,
                    local_keypoint_matches
                ])

        # --- !
                if release_count % 2 == 0:
                    #print('Test')
                    print("  radius = {} [G]: Done with {} releases ({}) ({} samples). Time to match {:.3f} seconds. ({})".format(
                        radius,
                        len(g_iteration_scores),
                        release_count,
                        len(conservative_scores),
                        time.time()-t0,
                        growing_p_point_cloud.shape
                    )
                         )
                    t0 = time.time()

                    current_errors = NN_matcher(g_iteration_scores)
                    print("   ({} actual planes) G_Error Rate: {}".format(
                        len(gen_planes),
                        np.sum(current_errors[:,1]/len(g_iteration_scores))
                    ))

                g_iteration_errors = NN_matcher(g_iteration_scores)
            try:
                print(radius,plane_number,len(conservative_scores),"G_Error Rate:",np.sum(g_iteration_errors[:,1]/len(g_iteration_scores)))
            except:
                pass

            per_plane_scores.append([
                plane_number,
                g_iteration_scores
            ])

            per_plane_errors.append([
                plane_number,
                g_iteration_errors
            ])

        conservative_scores.append([
            obj_,
            per_plane_scores
        ])

        conservative_errors.append([
            obj_,
            per_plane_errors
        ])
        
        if len(conservative_scores) % 5 == (samples-1)%5 :
            try:
                print("Radius = {}, {} objects done".format(radius,len(g_successive_scores)))
            except:
                pass
            
        with open('testing_results/conservative/radius_{}_RANSAC_conservative_scores.pickle'.format(radius), 'wb') as f:
            pickle.dump(conservative_scores,f)

        with open('testing_results/conservative/radius_{}_RANSAC_conservative_errors.pickle'.format(radius), 'wb') as f:
            pickle.dump(conservative_errors,f)
                

# 3.2 Results with conservative plane releasing

In [ ]:
conservative_RANSAC_error_results = []

t0 = time.time()

for radius in radius_range:
    
    succ_RansacGeneralizedNN_per_iteration_errors = []
    
    try:
                
        with open('testing_results/conservative/radius_{}_RANSAC_conservative_scores.pickle'.format(radius), 'rb') as f:
            conservative_scores = pickle.load(f)
        
        for obj_, per_plane_scores in conservative_scores:#[:-1]:  
            
            per_plane_errors = []
            
            skipped= False
            
            for max_plane, iteration_scores in per_plane_scores:
                
                iteration_errors = NN_matcher(iteration_scores)
                
                if iteration_errors.shape[0] >= int(releases/skip):
                    per_plane_errors.append(iteration_errors[:int(releases/skip)])
                else:
                    skipped = True
                    #print("RANSAC: skipped",iteration_errors.shape)
                    
            if not skipped:
                succ_RansacGeneralizedNN_per_iteration_errors.append(per_plane_errors)
       
        conservative_RANSAC_error_results.append([
            radius,
            np.asarray(succ_RansacGeneralizedNN_per_iteration_errors)
        ])
        
        print(np.asarray(succ_RansacGeneralizedNN_per_iteration_errors).shape)
              
    except Exception as ex:
        print(radius,": conservative RansacNN\n  ", ex)
        pass
    
    
    print("Done with radius = {:.2f} in {:.3f} seconds".format(radius,time.time() - t0))
    t0 = time.time()
    
"""
# Uncomment below if you want to overwrite the existing results.
"""
#with open('testing_results/conservative/conservative_RANSAC_error_results.pickle', 'wb') as f:
#    pickle.dump(conservative_RANSAC_error_results,f)

In [ ]:
"""

Preparing the results of the case with *Conservative Releasing*.

"""

releases_range = np.arange(1,releases-skip,skip)

X, Y = np.meshgrid(releases_range, planes)

test_vp_cn_05 = np.asarray(conservative_RANSAC_error_results[0][1])
mean_vp_cn_05 = np.mean(test_vp_cn_05[:,:,:,1],axis = 0)

#test_vp_cn_10 = np.asarray(conservative_RANSAC_error_results[1][1])
#mean_vp_cn_10 = np.mean(test_vp_cn_10[:,:,:,1],axis = 0)

intra_vp_cn_05 = np.zeros(test_vp_cn_05.shape[1:])
#intra_vp_cn_10 = np.zeros(test_vp_cn_10.shape[1:])

for plane_i, plane in enumerate(planes):
        
    for rel_i, rel in enumerate(releases_range):
    
        correct_interspace_labels_idxs_05 = np.where(test_vp_cn_05[:,plane_i,rel_i,1]==0)[0]
        #correct_interspace_labels_idxs_10 = np.where(test_vp_cn_10[:,plane_i,rel_i,1]==0)[0]

        intraspace_errors_05  = test_vp_cn_05[correct_interspace_labels_idxs_05,plane_i,rel_i,2]
        #intraspace_errors_10  = test_vp_cn_10[correct_interspace_labels_idxs_10,plane_i,rel_i,2]
        
        intra_vp_cn_05[plane_i,rel_i] = np.asarray([
            np.mean(intraspace_errors_05),
            np.std(intraspace_errors_05),
            0,
            np.nan
        ])
        

In [ ]:
fig = plt.figure(figsize=(11,8))
ax = plt.axes(projection='3d')

surf = ax.plot_surface(
    X, Y, 
    mean_vp_cn_05, 
    cmap=plt.cm.plasma,
)
surf.set_clim(0.0,1.0)

ax.set_title("r = 0.5", fontsize = 24)
ax.set_xlabel("Releases", labelpad=10, fontsize = 24)
ax.set_xlim(0,releases)
ax.set_xticklabels(releases_range,fontsize = 16)
ax.set_zlabel("INTER-space Privacy", labelpad=10, fontsize = 24)
ax.set_zlim(0,1)
ax.set_zticklabels([0,0.2,0.4,0.6,0.8,1.0],fontsize = 16)
ax.set_ylabel("Max number of planes", labelpad=10, fontsize = 22)#, offset = 1)
ax.set_ylim(0,30)
ax.set_yticklabels(np.arange(0,35,5),fontsize = 16)

cbar = fig.colorbar(surf, aspect=30, ticks = np.arange(0.0,1.1,0.25))
cbar.ax.set_yticklabels(np.arange(0.0,1.1,0.25),fontsize = 16)

ax.view_init(25,135);